# Jeu d'apprentissage uniquement multi-classes ☕️


**<span style='color:blue'> Objectifs de la séquence</span>** 
* Être sensibilisé&nbsp;:
    * à l'idée de prédiction d'ensembles.
* Être capable&nbsp;:
    * de formaliser la prédiction *average-K*,
    * d'implémenter un *average-K* avec $\texttt{pytorch}$.



 ----
Dans la plupart des données et y-compris sur des problèmes clairement "ensemblistes", nous ne disposons généralement QUE d'une seule classe par exemple d'apprentissage. Le problème n'est pas tant qu'il n'y en ait qu'une mais que nous n'avons AUCUN exemple d'absence. Ainsi, nous pouvons construire un problème d'optimisation qui récompense les prédictions d'ensembles contenant la bonne réponse mais qui ne ferait rien dans le cas contraire. Le problème est que finalement, en faisant cela, il suffit que notre modèle retourne toujours toutes les classes pour minimiser notre *loss*. Hors, ce n'est pas ce que nous souhaitons.

Soit $\mathcal{X}$ notre espace d'entrée, $\mathcal{Y}=\{1, \ldots, C\}$ celui des prédictions et notons $S_n=\{(X_i, Y_i)\}_{i\leq n}$ notre jeu de données sur $\mathcal{X}\times\mathcal{Y}$. Nous traitons bien sûr ici d'un problème de classification (cf. $\mathcal{Y}$). Comme c'est presque tout le temps le cas, chaque exemple d'apprentissage est associé à un unique label et nous souhaiterions construire un modèle capable de prédire un ensemble plutôt qu'un seul élément. Nous cherchons ainsi des applications :

$$h:\mathcal{X}\rightarrow\mathcal{P}(\mathcal{Y}).$$

Cette séquence se concentrera sur deux stratégies : les prédictions *Top-K* et les prédictions *Average-K*. Nous formaliserons le problème d'optimisation qu'elles cherchent à résoudre, nous en déduirons le classifieur de Bayes et nous illustrerons ces stratégies via des exemples.

## I. Prédiction *top-K*


### A. Une introduction théorique

L'approche la plus naïve consiste à se dire que s'il y a de l'ambiguïté dans nos données, alors il de ne pas prédire qu'une seule classe mais $K$ classes à chaque fois. Le problème n'est pas nécessairement évident et nous situerons donc dans un cadre particulier où notre modèle est un estimateur de la probabilité conditionnelle de chaque classe :

$$\hat{\eta}_k(x)=\mathbb{P}(Y=k|X=x),$$

sachant que par définition de la notion de probabilité, nous avons bien sûr :

$$\sum_k \hat{\eta}_k(x)=1.$$

Notre objectif est donc de construire un classifieur $\mathcal{S}:\mathcal{X}\rightarrow\mathcal{P}(\mathcal{Y})$ tel que $|\mathcal{S}(x)|=K,\ \forall x\in\mathcal{X}$. Nous cherchons ici à minimiser l'erreur suivante :

$$R(\mathcal{S})=\mathbb{P}(Y\not\in \mathcal{S}(X)).$$


**<span style='color:blue'> Question</span>** **À chaque prédiction exactement $K$ classes sont retournées. Cette fois-ci, nos prédictions se font dans l'espace $\mathcal{P}(\mathcal{Y})$ l'ensemble des parties de $\mathcal{Y}$. Notons $\eta_k(x)=\mathbb{P}(Y=k|X=x)$. Trouvez le classifieur de Bayes**


 ----


### B. En pratique



En pratique, on peut chercher à optimiser une *loss* directement dédiée au *Top-K* ou, à l'inverse, optimiser un modèle avec une *loss* classique permettant d'avoir un estimateur $\hat{\eta}$ et ensuite d'utiliser la règle de classification sur cet estimateur. C'est la seconde approche que nous choisissons ici.

In [ ]:
import torch
from torch.utils.data import DataLoader 
from torchvision import datasets
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose(
  [
      transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
  ]
)

cifar_train = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
cifar_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

batch_size = 128

trainloader = DataLoader(
  cifar_train, batch_size=batch_size
)

testloader = DataLoader(
  cifar_test, batch_size=batch_size, shuffle=True
)

print('Nb test batchs:', len(testloader))

Notre estimateur de la probabilité conditionnelle $\hat{\eta}$ est tout simplement notre réseau de neurones. Notez cependant que les sorties ne sont pas normalisées car, pour des raisons de stabilité, nous préférons laisser le soin de la normalisation à la *loss* *cross-entropy*. Ce n'est pas un problème car la normalisation se fait par la fonction :

$$\texttt{softmax}_k(z)=\frac{e^{z_k}}{\sum_j e^{z_j}},$$

qui est monotone.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.fc1 = nn.Linear(32 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc(x)
        return x

In [ ]:
model = Net()

In [ ]:
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR

In [ ]:
#Choose the loss function
criterion = nn.CrossEntropyLoss()

#Optimizer
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9, weight_decay=0.)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
loss_history = []
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:  # print every 2000 mini-batches
            print('\r[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000), end='')
            loss_history.append(running_loss / 2000)
            running_loss = 0.0
print('\r**** Finished Training ****')
plt.figure(figsize=(12, 8))
plt.plot([i for i in range(1, len(loss_history)+1)], loss_history, 
         label='My set-valued model')
plt.legend()
plt.show()

**<span style='color:blue'> Exercice</span>** **Adaptez le code suivant afin que votre modèle soit évalué avec une approche *Top-K*.**


 ----

In [ ]:
####### Complete this part ######## or die ####################
def test(model, loader, k, title, timeout=5):
    model.eval()  # on passe le modele en mode evaluation
    correct = 0
    total = 0
    set_size = 0
    with torch.no_grad():
        for i, data in enumerate(loader):
            images, labels = data
            outputs = model(images)

            correct += (outputs>threshold).float().gather(1, labels.view(-1,1)).sum()
            set_size += (outputs>threshold).float().sum()
            total += labels.size(0)
            if i >= timeout:
                break

    model.train()  # on remet le modele en mode apprentissage
    return correct / total, set_size / total
###############################################################


## II. Prédiction *Average-K*

### A. Une introduction théorique
On sent cependant assez rapidement que l'approche *Top-K* n'est pas totalement satisfaisante. On souhaiterait par exemple ne retourner qu'un seul élément lorsque la classe de l'image est évidente et simple et plus de $K$ éléments lorsque l'image est difficile à identifier. Nous cherchons toujours ici à minimiser l'erreur :

$$R(\mathcal{S})=\mathbb{P}(Y\not\in \mathcal{S}(X)).$$


La différence principale est la contrainte que l'on souhaite imposer sur notre modèle. Le classifieur de Bayes est ainsi la solution du problème :

$$\mathcal{S}^\star \in \text{argmin}_{\mathcal{S}}\mathbb{P}\big(Y\not\in \mathcal{S}(X)\big),\ \text{s.t. }\mathbb{E}\big[|S(X)|\big]\leq K,$$

où la première appartenance vient du fait que le classifieur de Bayes n'est pas nécessairement unique. En reprenant la formulation avec pénalité, $\exists\lambda > 0$ tel que :

$$\mathcal{S}^\star_\lambda \in \text{argmin}_{\mathcal{S}}\mathbb{P}\big(Y\not\in \mathcal{S}(X)\big)+\lambda\mathbb{E}\big[|S(X)|\big],$$

où on affiche la dépendance en $\lambda$. Il se trouve que ce problème possède une forme close que nous donnons immédiatement et qui revient à seuiller les probabilité :

$$\mathcal{S}_\lambda^\star(x)=\{k\in\{1, \ldots, C\}:\ \eta_k(x)\geq \lambda\}.$$


Il s'agit maintenant de choisir une valeur correcte de $\lambda$ en se rappelant que notre point de départ est notre souhait de retourner **en moyenne** $K$ éléments. Notons :

$$G(t)=\sum_{k=1}^C\mathbb{P}(\eta_k(X)\geq t).$$

C'est l'espérance de la taille de notre ensemble prédit si on seuil à $t$. Considérons son inverge généralisée, notée $G^{-1}(k)$. Notre classifieur de Bayes devient via ce dernier :

$$\mathcal{S}_\bar{k}^\star(x)=\{k\in\{1, \ldots, C\}:\ \eta_k(x)\geq G^{-1}(\bar{k})\}.$$

### B. En pratique

C'est bon, nous avons toutes les billes : 

*  On entraîne notre modèle sur un jeu d'apprentissage pour estimer $\hat{\eta}$,
*  On estime $G^{-1}(\bar{k})$ sur un jeu de validation.



In [ ]:
import torch
from torch.utils.data import DataLoader 
from torchvision import datasets
import torchvision.transforms as transforms

**Attention,** le code ci-dessous est très proche du code précédent, mais nous avons du créer un ensemble de validation à partir de notre jeu d'apprentissage.

In [ ]:
transform = transforms.Compose(
  [
      transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
  ]
)

cifar_train = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# Attention on split en train et val
train_set, val_set = torch.utils.data.random_split(cifar_train, [49000, 1000])

cifar_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

batch_size = 128

trainloader = DataLoader(
  train_set, batch_size=batch_size
)

valloader = DataLoader(
  val_set, batch_size=batch_size
)

testloader = DataLoader(
  cifar_test, batch_size=batch_size, shuffle=True
)

print('Nb test batchs:', len(testloader))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.fc1 = nn.Linear(32 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc(x)
        return x

In [ ]:
model = Net()

In [ ]:
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR

In [ ]:
#Choose the loss function
criterion = nn.CrossEntropyLoss()

#Optimizer
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9, weight_decay=0.)

In [ ]:
loss_history = []
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:  # print every 2000 mini-batches
            print('\r[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000), end='')
            loss_history.append(running_loss / 2000)
            running_loss = 0.0
print('\r**** Finished Training ****')
plt.figure(figsize=(12, 8))
plt.plot([i for i in range(1, len(loss_history)+1)], loss_history, 
         label='My set-valued model')
plt.legend()
plt.show()

**<span style='color:blue'> Exercice</span>** **Adaptez le code suivant afin que votre modèle soit évalué avec une approche *Average-K*.**


 ----

In [ ]:
####### Complete this part ######## or die ####################
def test(model, loader, k, title, timeout=5):
    model.eval()  # on passe le modele en mode evaluation
    correct = 0
    total = 0
    set_size = 0
    with torch.no_grad():
        for i, data in enumerate(loader):
            images, labels = data
            outputs = model(images)

            correct += (outputs>threshold).float().gather(1, labels.view(-1,1)).sum()
            set_size += (outputs>threshold).float().sum()
            total += labels.size(0)
            if i >= timeout:
                break

    model.train()  # on remet le modele en mode apprentissage
    return correct / total, set_size / total
###############################################################


**<span style='color:green'> Remarque</span>** 
La méthode *average-k* fonctionne *a priori* mieux, en ayant vu moins de données !



 ----